# Tutorial 02: Cell-Centered Finite Volume Method

This tutorial solves the same partial differential equation (PDE) as tutorial 01, namely
a nonlinear Poisson equation, with the following differences:

1. Implements a cell-centered finite volume method with two-point flux approximation as an example of a non-conforming scheme.
2. Implements *all* possible methods of a local operator.


# PDE Problem

Consider the following nonlinear Poisson equation (the same as in tutorial 01) with Dirichlet and Neumann boundary conditions:
\begin{equation} \label{eq:ProblemStrong}
\begin{aligned}
-\Delta u + q(u) &= f &&\text{in $\Omega$},\\
u &= g &&\text{on $\Gamma_D\subseteq\partial\Omega$},\\
-\nabla u\cdot \nu &= j &&\text{on $\Gamma_N=\partial\Omega\setminus\Gamma_D$}.
\end{aligned}
\end{equation}
$\Omega\subset\mathbb{R}^d$ is a domain, $q:\mathbb{R}\to\mathbb{R}$ is a given, possibly
nonlinear function and $f: \Omega\to\mathbb{R}$ is the source term and
$\nu$ denotes the unit outer normal to the domain.

# Cell-centered Finite Volume Method

The application of the cell-centered finite volume method as presented here is
restricted to *axiparallel meshes*. We assume that  the domain $\Omega$ is covered by a mesh
$\mathcal{T}_h = \{T_1, \ldots, T_M\}$ consisting of elements
which are closed sets satisfying
\begin{equation}
\bigcup_{T\in \mathcal{T}_h} T = \overline{\Omega}, \quad
\forall T, T' \in \mathcal{T}_h, T\neq T' : \mathring{T} \cap \mathring{T}' = \emptyset .
\end{equation}
In order to describe the method some further notation is needed.
The nonempty intersections $F = T_F^-\cap T_F^+$
of codimension 1 form the interior skeleton $\mathcal{F}_h^i=\{F_1,\ldots,F_N\}$.
Each intersection is equipped with a unit normal vector $\nu_F$ pointing from $T_F^-$ to $T_F^+$.
The intersections of an element $F=T_F^-\cap\partial\Omega$ with the domain
boundary form the set of boundary intersections $\mathcal{F}_h^{\partial\Omega}=
\{F_1,\ldots,F_L\}$ which can be further partitioned into
Dirichlet boundary intersections $\mathcal{F}_h^{\Gamma_D}$
and Neumann boundary intersections $\mathcal{F}_h^{\Gamma_N}$.
Each boundary intersection is equipped with a unit normal vector
$\nu_F$ which coincides with the unit outer normal to the domain.
Furthermore, $x_T$, $x_F$ denotes the center point of an element or face.
This notation is illustrated graphically in the Figure below.

![fvnotation](fv.png)
*Figure 1: Illustration of quantities associated with eements and intersections.*

For the cell-centered finite volume method the discrete function space involved
is the space of piecewise constant functions on the mesh:
\begin{equation*}
W_h = \{w\in L^2(\Omega) \,:\,  \text{$w|_T=$ const for all $T\in\mathcal{T}_h$}\} .
\end{equation*}

In order to derive the residual form we proceed as follows: multiply
equation \eqref{eq:ProblemStrong} with a test function $v\in W_h$, i.e. *from
the discrete space*, and use integration by parts:
\begin{align*}
\int_{\Omega} f v \,dx &= \int_{\Omega} [-\Delta u + q(u)] v\,dx\\
&= \sum_{T\in\mathcal{T}_h} v \int_T -\Delta u + q(u) \,dx &&\text{($v$ const on $T$)}\\
&= \sum_{T\in\mathcal{T}_h} \left[\int_T q(u) v \,dx - \int_{\partial T} \nabla u \cdot \nu v \,ds
\right] &&\text{(Gauss' thm.)} \\
&= \sum_{T\in\mathcal{T}_h} \int_T q(u) v \,dx
-\sum_{F\in\mathcal{F}_h^i} \int_F \nabla u \cdot \nu_F \bigl[v(x_{T_F^-}) - v(x_{T_F^+})\bigr] \,ds \\
& \hspace{10mm}-\sum_{F\in\mathcal{F}_h^{\partial\Omega}} \int_F \nabla u \cdot \nu_F \,ds .
&&\text{(rearrange)}
\end{align*}
At this point, the normal derivative $\partial_{\nu_F} u = \nabla u\cdot \nu_F$
is approximated by a difference quotient
\begin{equation*}
\nabla u\cdot \nu_F = \frac{u_h(x_{T_F^+})-u_h(x_{T_F^-})}{\|x_{T_F^+} - x_{T_F^-}\|}
 + \text{ error}
\end{equation*}
and all integrals are approximated by the midpoint rule
\begin{equation*}
\int_T f \,dx = f(x_T)|T| + \text{ error}
\end{equation*}
where $|T|$ is the measure of $T$.

Put together the cell-centered finite volume method can be stated
in its abstract form suitable for implementation in PDELab:
\begin{equation}
\boxed{ \text{Find $u_h\in W_h$ s.t.:} \quad r_h^{\text{CCFV}}(u_h,v) = 0 \quad \forall v \in W_h }
\end{equation}
where the residual form is
\begin{equation}
\label{eq:res_form_final}
\begin{split}
r_h^{\text{CCFV}}(u_h,v)
& = \sum_{T\in\mathcal{T}_h} q(u_h(x_T)) v(x_T) |T|
- \sum_{T\in\mathcal{T}_h} f(x_T) v(x_T) |T|\\
&\ - \sum_{F\in\mathcal{F}_h^i}
\frac{u_h(x_{T_F^+})-u_h(x_{T_F^-})}{\|x_{T_F^+} - x_{T_F^-}\|}
\bigl[v(x_{T_F^-}) - v(x_{T_F^+})\bigr] |F|\\
&\ + \sum_{F\in\mathcal{F}_h^{\partial\Omega}\cap\Gamma_D}
\frac{u_h(x_{T_F^-})}{\|x_{F} - x_{T_F^-}\|} v(x_{T_F^-}) |F| \\
&\ - \sum_{F\in\mathcal{F}_h^{\partial\Omega}\cap\Gamma_D}
\frac{g(x_{F})}{\|x_{F} - x_{T_F^-}\|} v(x_{T_F^-}) |F|
+ \sum_{F\in\mathcal{F}_h^{\partial\Omega}\cap\Gamma_N} j(x_{F}) v(x_{T_F^-}) |F| .
\end{split}
\end{equation}
In this case *five* different types of integrals are involved in the
residual form:

1. Volume integral depending on trial and test function.
2. Volume integral depending on test function only.
3. Interior intersection integral depending on trial and test function.
4. Boundary intersection integral depending on trial and test function.
5. Boundary intersection integral depending on test function only.

Also note that no constraints on the function space are necessary in this case.
Dirichlet as well as Neumann boundary conditions are built weakly into the
residual form!

Finally, many types of discontinuous Galerkin finite element methods (DGFEM)
lead to the same five types of integrals and can be applied on general unstructured
conforming as well as nonconforming meshes.

## General Residual Form

The residual form of the cell-centered finite volume method suggests that
all residual forms could be composed of five different types of terms
in the following way:
\begin{equation}
\begin{split}
r(u,v) &=
\sum_{T\in\mathcal{T}_h} \alpha_T^V(R_T u, R_T v)
+ \sum_{T\in\mathcal{T}_h} \lambda_T^V(R_T v) \\
&\qquad+ \sum_{F\in\mathcal{F}_h^i} \alpha_F^S(R_{T_F^-} u,R_{T_F^+} u, R_{T_F^-} v, R_{T_F^+} v)\\
&\qquad+ \sum_{F\in\mathcal{F}_h^{\partial\Omega}} \alpha_F^B(R_{T_F^-} u, R_{T_F^-} v)
+ \sum_{F\in\mathcal{F}_h^{\partial\Omega}} \lambda_F^B(R_{T_F^-} v) .
\end{split}\label{eq:GeneralResidualForm}
\end{equation}
Here, we define the restriction
of a function $u\in U$ to an element by
\begin{equation*}
(R_T u)(x) = u(x) \quad \forall x\in\mathring{T} .
\end{equation*}
Note that the restriction of a function to element $T$ is only defined in
the interior of $T$. On interior intersections $F$, functions may be two-valued
and limits from within the elements $T_F^-, T_F^+$ need to be defined
(when $U$ is the space of element-wise constants that is trivial).

The five terms comprise volume integrals (superscript $V$), interior skeleton integrals
(superscript $S$) and boundary integrals (superscript $B$). Furthermore, the
$\alpha$-terms depend on trial and test functions whereas the $\lambda$-terms only
depend on the test function and involve the data of the PDE.

Each of the five terms $\alpha_T^V$, $\alpha_F^S$, $\alpha_F^B$,
$\lambda_T^V$, $\lambda_F^B$ corresponds to one method on the
local operator.
In addition to the evaluation of residuals also Jacobians and
matrix-free application of Jacobians are needed. This gives rise
to in total $5+3+3=11$ possible methods on a local operator given in the following table:

|    | volume | skeleton | boundary |
|----|----|-----|-----|
|  residual  | `alpha_volume`  <br> `lambda_volume` |`alpha_skeleton` | `alpha_boundary` <br> `lambda_boundary`|
|  Jacobian  | `jacobian_volume`       |`jacobian_skeleton`        | `jacobian_boundary`        |
|  Jac. app. | `jacobian_apply_volume` |`jacobian_apply_skeleton`  | `jacobian_apply_boundary`  |

# Realization in PDELab

The instantiation of the grid is very similar to tutorials 00 and 01 and thus is not explained in detail here.

In [ ]:
#include <dune/jupyter.hh>
#include "problem.hh" 
#include "nonlinearpoissonfv.hh"

In [ ]:
// open ini file
Dune::ParameterTree ptree;
Dune::ParameterTreeParser ptreeparser;
ptreeparser.readINITree("tutorial02.ini",ptree);

const int dim = 2;
const int refinement = ptree.get<int>("grid.refinement");

In [ ]:
using Grid = Dune::YaspGrid<dim>;

// define the extensions of the domain: a unit square with N by N cells
Dune::FieldVector<double,dim> lowerleft(0.0);
Dune::FieldVector<double,dim> upperright(1.0);
auto N = Dune::filledArray<dim, unsigned int>(25);

// build a structured grid
auto grid = Dune::StructuredGridFactory<Grid>::createCubeGrid(lowerleft, upperright, N);
grid->globalRefine(refinement);
using GV = Grid::LeafGridView;
GV gv = grid->leafGridView();

Also the rest is very similar in structure to tutorial 00 and 01. Here we just point out the differences.

In [ ]:
using DF = GV::Grid::ctype;           // type for ccordinates
using RF = double;                   // type for computations

// make user functions
RF eta = ptree.get("problem.eta",(RF)1.0);
Problem<RF> problem(eta);
auto g = Dune::PDELab::makeGridFunctionFromCallable(
    gv,
    [&](const auto& e, const auto& x){
        return problem.g(e,x);
    }
);;

The cell-centered finite volume method is based on the space of piecewise constant functions on the mesh $W_h$. The following code segment constructs this function space using the class `P0LocalFiniteElementMap`:

In [ ]:
// Make grid function space
using FEM = Dune::PDELab::P0LocalFiniteElementMap<DF,RF,dim>;
FEM fem(Dune::GeometryTypes::cube(dim));
using CON = Dune::PDELab::NoConstraints;
using VBE = Dune::PDELab::ISTL::VectorBackend<>;
using GFS = Dune::PDELab::GridFunctionSpace<GV,FEM,CON,VBE>;
GFS gfs(gv,fem);
gfs.name("Q0");

The following is the same as before:

In [ ]:
// A coefficient vector
using Z = Dune::PDELab::Backend::Vector<GFS,RF>;
Z z(gfs); // initial value

// Make a grid function out of it
using ZDGF = Dune::PDELab::DiscreteGridFunction<GFS,Z>;
ZDGF zdgf(gfs,z);

// Fill the coefficient vector
Dune::PDELab::interpolate(g,gfs,z);

// Make a local operator
using LOP = NonlinearPoissonFV<Problem<RF> >;
LOP lop(problem);

The constraints class `NoConstraints` is used to express that there are no constraints on the function space.
Now no constraints container type is exported by the grid function space. Instead the class `EmptyTransformation` is used in the grid operator:

In [ ]:
 // Make a global operator
using MBE = Dune::PDELab::ISTL::BCRSMatrixBackend<>;
MBE mbe(2*dim+1); // guess nonzeros per row
using CC = Dune::PDELab::EmptyTransformation;
using GO = Dune::PDELab::GridOperator<
  GFS,GFS,  /* ansatz and test space */
  LOP,      /* local operator */
  MBE,      /* matrix backend */
  RF,RF,RF, /* domain, range, jacobian field type*/
  CC,CC     /* constraints for ansatz and test space */
  > ;
GO go(gfs,gfs,lop,mbe);

In [ ]:
// Select a linear solver backend
using LS = Dune::PDELab::ISTLBackend_SEQ_CG_AMG_SSOR<GO>;
LS ls(100,2);

// solve nonlinear problem
Dune::PDELab::Newton<GO,LS,Z> newton(go,z,ls);
newton.setReassembleThreshold(0.0);
newton.setVerbosityLevel(3);
newton.setReduction(1e-10);
newton.setMinLinearReduction(1e-4);
newton.setMaxIterations(25);
newton.setLineSearchMaxIterations(10);
newton.apply();

Cell-wise data is passed to the `VTKWriter` using its method `addCellData`:

In [ ]:
// Write VTK output file
Dune::VTKWriter<GV> vtkwriter(gv,Dune::VTK::conforming);
using VTKF = Dune::PDELab::VTKGridFunctionAdapter<ZDGF> ;
vtkwriter.addCellData(std::shared_ptr<VTKF>(new
                                      VTKF(zdgf,"fesol")));

These are the only changes to the driver!

In [ ]:
vtkwriter

## The `Problem` Class

The class `NonlinearPoissonFV` explained below uses the same problem class as the class `NonlinearPoissonFEM`. This means that the same problem can be easily solved using the two different methods.

## Local Operator `NonlinearPoissonFV`

The class `NonlinearPoissonFV` implements the element-wise computations of the cell-centered finite volume method. In particular, it provides a full implementation of all possible methods on a local operator including analytic Jacobians. The class has the problem class as a template parameter:

```c++
template<typename Param>
class NonlinearPoissonFV :
  public Dune::PDELab::FullVolumePattern,
  public Dune::PDELab::FullSkeletonPattern,
  public Dune::PDELab::LocalOperatorDefaultFlags
```

The base class `FullSkeletonPattern` provides the local operator with a method coupling all degrees of freedom of two elements sharing an intersection.
In combination with `FullVolumePattern` this provides the sparsity pattern of the matrix.

The only private data member is a reference to an object to the parameter class:

```c++
 Param& param;        // parameter functions
```

The public section begins with a definition of flags controlling assembly of the sparsity pattern

```c++
  // pattern assembly flags
  enum { doPatternVolume = true };
  enum { doPatternSkeleton = true };
```

as well as element contributions:

```c++
 // residual assembly flags
  enum { doLambdaVolume = true };
  enum { doLambdaBoundary = true };
  enum { doAlphaVolume = true };
  enum { doAlphaSkeleton  = true };
  enum { doAlphaBoundary  = true };
```

These five flags specify that all five contributions will be provided. The constructor just gets a reference of the parameter object:

```c++
  NonlinearPoissonFV (Param& param_)
```

### Method `lambda_volume`

This method was already present in the finite element method aand corresponds
to sum number two on the right hand side of equation \eqref{eq:res_form_final}. <font color = 'red'> X </font>.
The element contributions for the cell-centered finite volume method are particularly simple to implement. Here is the right hand side contribution:

$$ \lambda_T^V(R_T v) = - f(x_T) v(x_T) |T| $$

```c++  
  template<typename EG, typename LFSV, typename R>
  void lambda_volume (const EG& eg, const LFSV& lfsv,
                      R& r) const
  {
    // center of reference element
    auto cellgeo = eg.geometry();
    auto cellcenterlocal =
      referenceElement(cellgeo).position(0,0);

    // accumulate residual
    auto f = param.f(eg.entity(),cellcenterlocal);
    r.accumulate(lfsv,0,-f*cellgeo.volume());
  }
```

Note that throughout the whole class we assume that the basis functions of the space $W_h$ are one on one element and zero on all others, i.e.
\begin{equation}
\phi_i(x) = \left\{ \begin{array}{ll} 1 & x\in T_i \\ 0 & \text{else} \end{array}
\right . .
\end{equation}
This means that basis functions will never be evaluated!

### Method `lambda_boundary`

This method was also already present in the finite element method
and corresponds to sums five \textit{and} six on the right hand side of equation \eqref{eq:res_form_final}.<font color = 'red'> X </font>.
$$ \lambda_F^B(R_{T_F^{-}v}) = -
\frac{g(x_{F})}{\|x_{F} - x_{T_F^-}\|} v(x_{T_F^-}) |F| +  j(x_{F}) v(x_{T_F^-}) |F| $$
It assembles contributions from Dirichlet and Neumann
boundary conditions and has the interface

```c++
  template<typename IG, typename LFSV, typename R>
  void lambda_boundary (const IG& ig, const LFSV& lfsv_i,
                        R& r_i) const
  {
```   

First  the center of the reference element of the intersection is extracted and the boundary condition type is evaluated:

```c++
    // face volume for integration
    auto facegeo = ig.geometry();
    auto facecenterlocal =
      referenceElement(facegeo).position(0,0);

    // evaluate boundary condition and quit on Dirichlet
    bool isdirichlet =
      param.b(ig.intersection(),facecenterlocal);

```

Now comes the part for the Dirichlet boundary conditions where we need to compute the distance from the face center to the element center,
the value of the Dirichlet boundary condition and the measure of the face:

```c++
if (isdirichlet)
      {
        // inside cell center
        auto insidecenterglobal=ig.inside().geometry().center();

        // face center in global coordinates
        auto facecenterglobal = facegeo.center();

        // compute distance of these two points
        insidecenterglobal -= facecenterglobal;
        auto distance = insidecenterglobal.two_norm();

        // face center in local coordinates of the element
        auto facecenterinelement=ig.geometryInInside().center();

        // evaluate Dirichlet condition
        auto g = param.g(ig.inside(),facecenterinelement);

        // face volume for integration
        auto face_volume = facegeo.volume();

        // contribution to residual
        r_i.accumulate(lfsv_i,0,-g/distance*face_volume);
      }
```

The Neumann part is much simpler:

```c++
  else
      {
        // contribution to residual from Neumann boundary
        auto j = param.j(ig.intersection(),facecenterlocal);
        r_i.accumulate(lfsv_i,0,j*facegeo.volume());
      }
```

### Method `alpha_volume`

Now `alpha_volume` has also been presented before and corresponds to the first sum on the right hand side of equation \eqref{eq:res_form_final}. 
$$ \alpha_T^V(R_Tu,R_Tv) = q(u_h(x_T)) v(x_T) |T| $$
Here it just contains the evaluation of the reaction term with the midpoint rule:

```c++
 template<typename EG, typename LFSU, typename X,
           typename LFSV, typename R>
  void alpha_volume (const EG& eg, const LFSU& lfsu, const X& x,
                     const LFSV& lfsv, R& r) const
  {
    // get cell value
    auto u = x(lfsu,0);

    // evaluate reaction term
    auto q = param.q(u);

    // and accumulate
    r.accumulate(lfsv,0,q*eg.geometry().volume());
  }
```

### Method `jacobian_volume`

Now we come to the first method that has not been implemented in previous examples. The method `jacobian_volume` will assemble the entries of the Jacobian coupling all degrees of the given element. As there is only one degree of freedom per element there is only one matrix entry to assemble. The matrix entries are returned in the container which is the last argument of the method:

```c++  
  template<typename EG, typename LFSU, typename X,
           typename LFSV, typename M>
  void jacobian_volume (const EG& eg, const LFSU& lfsu, const X& x,
                        const LFSV& lfsv, M& mat) const
```

First the derivative of the nonlinearity is evaluated

```c++
    auto u = x(lfsu,0);
    auto qprime = param.qprime(u);
```

and the matrix entry is written into the container

```c++
    mat.accumulate(lfsv,0,lfsu,0,qprime*eg.geometry().volume());
```

`mat.accumulate` has five arguments:
the *matrix row* given by local test space and number of the test function, the *matrix column* given by local trial space and number of the trial function and, as last argument, the contribution to the matrix entry which is added to the global Jacobian matrix.

### Method `jacobian_apply_volume`

This method is very similar to the previous method except that
it multiplies the local Jacobian contribution immediately with a vector
and accumulates the result.

The method has the following interface:

```c++
  template<typename EG, typename LFSU, typename X,
           typename LFSV, typename R>
  void jacobian_apply_volume (const EG& eg, const LFSU& lfsu,
                              const X& x, const X& z,
                              const LFSV& lfsv, R& r) const
```

`x` are the coefficients of the linearization point and `z` are the entries of the vector to be multiplied with the Jacobian. The result is accumulated to the container `r`. Here is the implementation:

```c++
  // evaluate derivative reaction term
    auto u = x(lfsu,0);
    auto qprime = param.qprime(u);

    // and accumulate
    r.accumulate(lfsv,0,qprime*z(lfsu,0)*eg.geometry().volume());
```

Comparison with `jacobian_volume` shows that the Jacobian entry is multiplied with the entry of `z`.

### Method `alpha_skeleton`

This is the major new method needed to implement the flux terms
in finite volume and discontinuous Galerkin methods. Note that `alpha_skeleton` needs to assemble
residual contributions for all test functions involved with *both*
elements next to the intersection and corresponds to sum number three on the right hand side of equation \eqref{eq:res_form_final}.

 $$ \alpha_F^S(R_{T_F^-}u,R_{T_F^+}u,R_{T_F^-}v,R_{T_F^+}v) =- \
\frac{u_h(x_{T_F^+})-u_h(x_{T_F^-})}{\|x_{T_F^+} - x_{T_F^-}\|}
\bigl[v(x_{T_F^-}) - v(x_{T_F^+})\bigr] |F| $$   
It has the following interface:

```c++
  template<typename IG, typename LFSU, typename X,
           typename LFSV, typename R>
  void alpha_skeleton (const IG& ig,
         const LFSU& lfsu_i, const X& x_i, const LFSV& lfsv_i,
         const LFSU& lfsu_o, const X& x_o, const LFSV& lfsv_o,
         R& r_i, R& r_o) const
```

The arguments comprise an intersection, local trial function and local test space for both elements adjacent to the intersection
and containers for the local residual contributions in both elements.
The subscripts `_i` and `_o` correspond to "inside" and "outside". W.r.t. our notation above "inside" corresponds to "-" and "outside" corresponds to "+".

It starts by extracting the elements adjacent to the intersection

```c++
    auto cell_inside = ig.inside();
    auto cell_outside = ig.outside();
```

and then extracts their geometries

```c++
    auto insidegeo = cell_inside.geometry();
    auto outsidegeo = cell_outside.geometry();
```

and the centers

```c++
    auto inside_global = insidegeo.center();
    auto outside_global = outsidegeo.center();
```

Now the distance of the centers can be computed

```c++
    inside_global -= outside_global;
    auto distance = inside_global.two_norm();
```

and the measure of the face is extracted

```c++
    auto facegeo = ig.geometry();
    auto face_volume = facegeo.volume();
```

which puts us in the position to accumulate the residual contributions

```c++
    auto dudn = (x_o(lfsu_o,0)-x_i(lfsu_i,0))/distance;
    r_i.accumulate(lfsv_i,0,-dudn*face_volume);
    r_o.accumulate(lfsv_o,0, dudn*face_volume);
```

In fact, the contribution to the inside element, i.e. to `r_i` is the flux from the inside to the outside element. The contribution to the outside element residual is exactly the negative value, i.e. we have local conservation.

### Method `jacobian_skeleton`

In the computation of the Jacobian w.r.t. skeleton terms we can exploit the fact that the discrete residual form is actually linear in these terms as the nonlinearity is restricted to the volume term only.

An interior face contributes to four matrix parts of the global matrix as there are test functions on the inside and outside elements (corresponding to rows in the matrix) as well as trial functions on the inside and outside element (corresponding to columns of the matrix). In the case of the cell-centered finite volume method for the nonlinear Poisson equations there is only one degree of freedom and test function per element, so there are four matrix entries which the face contributes. In case of higher order discontinuous Galerkin schemes and/or systems of PDEs there are four blocks of the matrix where the face contributes to. The following figure illustrates the matrix structure and the corresponding submatrices. For ease of drawing it is assumed that all trial and test functions of one element are numbered consecutively but this need not be the case!

![jacobianskeleton](jacobianSkeleton.png)
*Figure 1: Matrix block contributions computed by `jacobian_skeleton`*

The method has the following interface:
```c++
  template<typename IG, typename LFSU, typename X,
           typename LFSV, typename M>
  void jacobian_skeleton (const IG& ig,
         const LFSU& lfsu_i, const X& x_i, const LFSV& lfsv_i,
         const LFSU& lfsu_o, const X& x_o, const LFSV& lfsv_o,
         M& mat_ii, M& mat_io,
         M& mat_oi, M& mat_oo) const
  {
```

It is very similar to `alpha_skeleton` except that four containers are passed where the matrix entries of the four blocks need to be stored. The computation of distance of cell centers and the face volume are
exactly the same as in `alpha_skeleton`. Then the matrix entries are given by:

```c++
   mat_ii.accumulate(lfsv_i,0,lfsv_i,0, face_volume/distance);
   mat_io.accumulate(lfsv_i,0,lfsv_o,0,-face_volume/distance);
   mat_oi.accumulate(lfsv_o,0,lfsv_i,0,-face_volume/distance);
   mat_oo.accumulate(lfsv_o,0,lfsv_o,0, face_volume/distance);
```

### Method `jacobian_apply_skeleton`

The `jacobian_apply_skeleton` method needs to compute the local Jacobian contributions and multiply them with a given coefficient vector. It has the following interface

```c++
  template<typename IG, typename LFSU, typename X, typename LFSV,
           typename Y>
  void jacobian_apply_skeleton
  ( const IG& ig,
    const LFSU& lfsu_i, const X& x_i, const X& z_i, const LFSV& lfsv_i,
    const LFSU& lfsu_o, const X& x_o, const X& z_o, const LFSV& lfsv_o,
    Y& y_i, Y& y_o) const
```

`x_i`, `x_o` are the linearization point and `z_i`, `z_o` are the coefficients to multiply with. As the skeleton terms are linear with respect to degrees of freedom the Jacobian does not depend on the linearization point and we may reuse the `alpha_skeleton` method here:

```c++
 alpha_skeleton(ig,lfsu_i,z_i,lfsv_i,lfsu_o,z_o,lfsv_o,y_i,y_o);
```

### Method `alpha_boundary`

The`alpha_boundary` method is also new. It corresponds
to the fourth sum on the right hand side of equation \eqref{eq:res_form_final}
$$ \alpha_F^B(R_{T_F^-}u,R_{T_F^-}v) =
\frac{u_h(x_{T_F^-})}{\|x_{F} - x_{T_F^-}\|} v(x_{T_F^-}) |F| $$
which is again linear in the degrees of freedom.
The interface is now:

```c++
  template<typename IG, typename LFSU, typename X,
           typename LFSV, typename R>
  void alpha_boundary (const IG& ig,
                       const LFSU& lfsu_i, const X& x_i,
                       const LFSV& lfsv_i, R& r_i) const
```
The residual contribution depends only on quantities on the inside element of the intersection.
First we need to check whether the face is on the Dirichlet boundary:

```c++
   auto facegeo = ig.geometry();
    auto facecenterlocal =
      referenceElement(facegeo).position(0,0);
    bool isdirichlet = param.b(ig.intersection(),facecenterlocal);
    if (!isdirichlet) return;
```

Then the distance from face center to cell center is computed:

```c++
   // inside cell center
    auto insidecenterglobal = ig.inside().geometry().center();

    // face center in global coordinates
    auto facecenterglobal = facegeo.center();

    // compute distance of these two points
    insidecenterglobal -= facecenterglobal;
    auto distance = insidecenterglobal.two_norm();
```

and the residual contribution can be accumulated:

```c++
    // face volume for integration
    auto face_volume = facegeo.volume();

    // contribution to residual
    r_i.accumulate(lfsv_i,0,x_i(lfsu_i,0)/distance*face_volume);
```

### Method `jacobian_boundary`

The`jacobian_boundary` method computes the Jacobian contributions resulting from boundary face integrals and has the following interface:
```c++
template<typename IG, typename LFSU, typename X,
           typename LFSV, typename M>
  void jacobian_boundary (const IG& ig,
                          const LFSU& lfsu_i, const X& x_i,
                          const LFSV& lfsv_i, M& mat_ii) const
```
The interface is the same as for `alpha_boundary` except that a matrix container is passed as the last argument.

As the contributions only depend on test and trial functions of the inside element there is only contribution to one matrix entry:

```c++
 mat_ii.accumulate(lfsv_i,0,lfsv_i,0,face_volume/distance);
```

### Method `apply_jacobian_boundary`

Finally, the `jacobian_apply_boundary` does a matrix-free Jacobian times vector multiplication. Due to linearity we can reuse the `alpha_boundary` method:
```c++
  template<typename IG, typename LFSU, typename X,
           typename LFSV, typename Y>
  void jacobian_apply_boundary
  ( const IG& ig,
    const LFSU& lfsu_i, const X& x_i, const X& z_i,
    const LFSV& lfsv_i, Y& y_i) const
  {
    // reuse alpha_boundary because it is linear
    alpha_boundary(ig,lfsu_i,z_i,lfsv_i,y_i);
  }
```